In [2]:
#! usr/bin/env python3
import praw
import pandas as pd
import datetime as dt

In [3]:
reddit = praw.Reddit(client_id='4qQrwmPQAVOwiw', \
                     client_secret='MVf28OrOH-Ak_-XLSlLHWqbfono', \
                     user_agent='API dan', \
                     username='blablabla123983', \
                     password='newguy')

In [4]:
subreddit = reddit.subreddit('Bitcoin')

In [5]:
top_subreddit = subreddit.top()

In [6]:
top_subreddit = subreddit.top(limit=500)

In [7]:
for submission in subreddit.top(limit=1):
    print(submission.title, submission.id)

It's official! 1 Bitcoin = $10,000 USD 7g9cd3


In [17]:
topics_dict = { "title":[], \
                "score":[], \
                "id":[], "url":[], \
                "comms_num": [], \
                "created": [], \
                "body":[]}


In [18]:
for submission in top_subreddit:
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(submission.selftext)

In [20]:
topics_data = pd.DataFrame(topics_dict)
topics_data

,body,comms_num,created,id,score,title,url
0,,4595,1.511948e+09,7g9cd3,48506,"It's official! 1 Bitcoin = $10,000 USD",https://v.redd.it/e7io27rdgt001
1,,791,1.517741e+09,7v438b,48480,The last 3 months in 47 seconds.,https://v.redd.it/typ8fdslz3e01
2,,3172,1.511701e+09,7fkqh5,42431,It's over 9000!!!,https://i.imgur.com/jyoZGyW.gifv
3,,804,1.515300e+09,7olruz,42055,Everyone who's trading BTC right now,http://cdn.mutually.com/wp-content/uploads/201...
4,,1243,1.512064e+09,7gla05,29318,I hope James is doing well,https://i.redd.it/h4ngqma643101.jpg
5,,1531,1.516123e+09,7qr4ym,29133,Weeeeeeee!,https://i.redd.it/iwl7vz69cea01.gif
6,,1180,1.517435e+09,7u9zbd,28322,Bitcoin.. The King,https://i.redd.it/4tl0oustqed01.jpg
7,,1321,1.513181e+09,7jht7c,28060,Nothing can increase by that much and still be...,https://i.imgur.com/oWePY7q.jpg
8,,918,1.511001e+09,7dpypn,26952,"This is why I want bitcoin to hit $10,000",https://i.redd.it/fhzsxgcv9nyz.jpg
9,,1505,1.518938e+09,7yaaz2,26327,Bitcoin Doesn't Give a Fuck.,https://v.redd.it/ty2y74gawug01


In [21]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

In [22]:
_timestamp = topics_data["created"].apply(get_date)

In [23]:
topics_data = topics_data.assign(timestamp = _timestamp)

In [26]:
topics_data.to_csv('bitcoindatanew.csv', index=False) 

In [25]:
topics_data

,body,comms_num,created,id,score,title,url,timestamp
0,,4595,1.511948e+09,7g9cd3,48506,"It's official! 1 Bitcoin = $10,000 USD",https://v.redd.it/e7io27rdgt001,2017-11-29 10:25:24
1,,791,1.517741e+09,7v438b,48480,The last 3 months in 47 seconds.,https://v.redd.it/typ8fdslz3e01,2018-02-04 11:42:03
2,,3172,1.511701e+09,7fkqh5,42431,It's over 9000!!!,https://i.imgur.com/jyoZGyW.gifv,2017-11-26 13:55:02
3,,804,1.515300e+09,7olruz,42055,Everyone who's trading BTC right now,http://cdn.mutually.com/wp-content/uploads/201...,2018-01-07 05:38:56
4,,1243,1.512064e+09,7gla05,29318,I hope James is doing well,https://i.redd.it/h4ngqma643101.jpg,2017-11-30 18:50:46
5,,1531,1.516123e+09,7qr4ym,29133,Weeeeeeee!,https://i.redd.it/iwl7vz69cea01.gif,2018-01-16 18:13:04
6,,1180,1.517435e+09,7u9zbd,28322,Bitcoin.. The King,https://i.redd.it/4tl0oustqed01.jpg,2018-01-31 22:46:59
7,,1321,1.513181e+09,7jht7c,28060,Nothing can increase by that much and still be...,https://i.imgur.com/oWePY7q.jpg,2017-12-13 17:02:51
8,,918,1.511001e+09,7dpypn,26952,"This is why I want bitcoin to hit $10,000",https://i.redd.it/fhzsxgcv9nyz.jpg,2017-11-18 11:25:57
9,,1505,1.518938e+09,7yaaz2,26327,Bitcoin Doesn't Give a Fuck.,https://v.redd.it/ty2y74gawug01,2018-02-18 08:19:24


In [ ]:
import logging
import settings
#CLI input
import argparse
import time

#reddit api (Python Reddit API Wrapper)
import praw
import os

import datetime

#JSON parsing
import json

import time
#Multithreading and CSV processing 
from multiprocessing import Process
import csv

#Setting default to UTF8 to deal with pesky ascii errors in python 2.x
import sys
reload(sys)
sys.setdefaultencoding("utf8")

#Change this to CSV writer
def saveSubmission(submission, filename):
    #Reddit accounts can be deleted, and then the username won't show. If it doesn't exist fill in [deleted]
    if not submission.author:
        name = '[deleted]'
    #Otherwise just assign the author name 
    else:
        name = submission.author.name

    #Write it to a CSV 
    csvwriter = csv.writer(open(filename, "a"))
    csvwriter.writerow([datetime.datetime.fromtimestamp(submission.created).strftime('%c'), submission.permalink,  name, submission.title, submission.selftext, submission.score])

# Downloads all the self posts from given subreddit
# ts_interval - the interval in seconds for each request cycle
# largest_timestamp - if not set, is set to 12 hours from now.
def getThreads(subredditName, ts_interval, ep1, ep2, user_ag, user):
    #Set filename for this subreddit. Timestamp it so everytime this script is run, results are appended to differnt file 
    filename = 'Reddit_'+subredditName+'_'+str(int(time.time()))+'.csv'

    #Creating header rows in the CSV file 
    csvwriter = csv.writer(open(filename, "a"))
    csvwriter.writerow(['timestamp', 'url',  'author', 'postTitle', 'postContent', 'score'])
    #Initialize the Reddit client with client ID, secret key, user agent (arbitrary), password and username
    #See the included readme for detailed explanation 
    reddit = praw.Reddit(client_id=user['clientID'],client_secret=user['secretKey'],user_agent=user_ag,password=user['password'],username=user['username'])
    print(reddit.user.me())
    #If no time epoch is specified, 
    if ep2 is None:
        ep2 = int(time.time()) + 12*3600

    #cts for current timestamp. Defines the sliding window between which the search query is executed. 
    cts2 = ep2
    #Substract the ts interval from the second timstamp to move the window
    cts1 = ep2 - ts_interval
    currentTSInterval = ts_interval
    while True:
        #Use reddit search API to search for posts between two timestamps (created above), on the subreddit defined in subRedditName
        #Using cloudsearch as syntax, because otherwise it is not possible to search between timestamps 
        try:
            searchResults = list(reddit.subreddit(subredditName).search('timestamp:{}..{}'.format(cts1, cts2), syntax='cloudsearch'))
        #If it errors for whatever reason, log the error and continue 
        except Exception as e:
            logging.exception(e)
            continue

        #logging.debug("Got {} submissions in interval {}..{}".format(len(searchResults), cts1, cts2))
        #It returns at most 100 search results, so if it maxes out, we split the interval and search again 
        #This way we ensure that we do not miss any posts 
        if len(searchResults) == 100:
            currentTSInterval /= 2
            cts1 = cts2 - currentTSInterval
           # logging.debug("Reducing time interval to {}".format(currentTSInterval))
            continue

        #Printing Reddit post title to show progress 
        for submission in searchResults:
            print (submission.title)
            saveSubmission(submission, filename)

        #Move the timestamp window by setting the second ts to the first, and decreasing the second one
        cts2 = cts1
        cts1 = max(ep1, cts2 - currentTSInterval)

        #Once the timestamps meet, the range is effectively closed and no more topics are to be collected 
        if cts1 == cts2:
            break

        if len(searchResults) <= 7:
            currentTSInterval *= 2
           # logging.debug("Increasing time interval to {}".format(currentTSInterval))

def main():
    parser = argparse.ArgumentParser(description='Parallel Subreddit Downloader')
    parser.add_argument("--subreddit", dest="subreddit", required=True, help="Download the whole subreddit")
    parser.add_argument("--timestamp_interval", dest="timestamp_interval", type=int, required=True)
    parser.add_argument("--year", dest="collect_year", type=int, required=True)
    argsx = parser.parse_args()
    collect_year = str(argsx.collect_year)


    #Timeframe between which to scrape
    tst1 = '01.01.'+collect_year+' 00:00:00'
    tst2 = '31.12.'+collect_year+' 23:59:59'
    pattern = '%d.%m.%Y %H:%M:%S'

    #Timezone definition, OS based
    os.environ['TZ'] = 'UTC'

    #Convert time string to epoch 
    epoch1 = int(time.mktime(time.strptime(tst1, pattern)))
    epoch2 = int(time.mktime(time.strptime(tst2, pattern)))

    #Split the increments over 8 different multithreading processes for increased speed 
    inc = (epoch2 - epoch1 + 1)/8
    ep1 = [0]*8
    ep2 = [0]*8
    processes = []

    for i in range(0, 8):
        ep1[i] = epoch1 + i*inc
        ep2[i] = epoch1 + (i + 1)*inc - 1
        user_agx = 'Collection' + str(i)
        user = settings.config
        p = Process(target=getThreads, args=(argsx.subreddit, argsx.timestamp_interval, ep1[i], ep2[i], user_agx, user))
        p.start()
        processes.append(p)

if __name__ == "__main__":
	start = time.time()
	main()
	end = time.time()
	print (end - start)
	
